In [1]:
%load_ext autoreload
%autoreload 2

import datetime
import pandas as pd

from opt_weights import *

In [2]:
start_date = datetime.date(2010,1,1)
end_date = datetime.date(2011,5,1)

In [3]:
import pandas.io.data as web
px = web.DataReader(["^GSPC", "^DJI", "^IXIC", "^FCHI", "^FTSE", "^N225"] , 'yahoo', start_date, end_date).Close
ret = px.div(px.shift()).dropna()

/Users/ryanchouck/dev/tf/venv/lib/python2.7/site-packages/pandas/io/data.py:35: FutureWarning: 
The pandas.io.data module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)


In [4]:
opt_weights = calc_opt_weights(ret)
opt_weights_reg = calc_opt_weights(ret, .01, 2)

In [5]:
opt_weights_reg

^DJI     3.711689e-01
^FCHI    2.907478e-08
^FTSE    2.521794e-01
^GSPC    1.615734e-01
^IXIC    1.148382e-06
^N225    2.150772e-01
dtype: float64

In [6]:
pd.DataFrame({'opt': (ret * opt_weights).sum(axis=1),
              'opt_reg': (ret * opt_weights_reg).sum(axis=1),
              'avg': ret.mean(axis=1)}).cumprod().tail(1)

,avg,opt,opt_reg
Date,,,
2011-04-28,1.075782,1.074864,1.068205


In [7]:
opt_weights = lambda x: calc_opt_weights(x, alpha=0, norm_type=2)
weights = rolling_fit_opt_weights(ret, opt_weights, look_ahead_per=1)

In [8]:
(ret * weights).sum(axis=1).cumprod().tail()

2011-04-19    1.001299
2011-04-20    1.014831
2011-04-21    1.014832
2011-04-27    1.014835
2011-04-28         NaN
dtype: float64

In [9]:
ret.mean(axis=1).cumprod().tail()

Date
2011-04-19    1.037701
2011-04-20    1.057262
2011-04-21    1.062134
2011-04-27    1.069385
2011-04-28    1.075782
dtype: float64

In [36]:
def cum_prod_grid(df, opt_weights_fun, max_look_ahead_per):
    end_date = df.index[-(max_look_ahead_per + 1)]
    alphas = {}
    for alpha in np.exp(np.linspace(-5, 5, 1)):
        pers = {}
        for per in (1,):
            weights = rolling_fit_opt_weights(ret, opt_weights, look_ahead_per=per)
            pers[per] = (ret * weights).sum(axis=1).cumprod()[:end_date].tail(1).values[0]
        alphas[alpha] = pd.Series(pers)
    return pd.DataFrame(alphas)

opt_weights_func = lambda x, y: calc_opt_weights(x, alpha=y, norm_type=2)
cum_prod_grid(ret, opt_weights_func, 10)

2011-04-11 00:00:00


,0.00673794699909
1,1.07826
